# Assignment 2 -> tubes
So, your task is to augment (expand) the dataset, for each category, atleast 10 images are required. Each image carries more than 2 MB memory, which is not at all feasible for deep learning. If you open the image, you will observe black background space around the test tube which is also needed to be eliminated by applying a suitable algorithm. 

In [222]:
import cv2
from PIL import Image
import numpy as np
import os
import fnmatch

In [245]:
class removeBlackBackground:
    def cropImage(self, imageX, imageXGray, bwMin, bwMax, threshold):     
        ret,tubeXbw = cv2.threshold(imageXGray,bwMin,bwMax,cv2.THRESH_BINARY)
        tubeXbwBlur=cv2.blur(tubeXbw,(5,5))

        colMeans=np.mean(tubeXbwBlur, axis=0)
        rowMeans=np.mean(tubeXbwBlur, axis=1)
        colThres=np.nonzero(colMeans > threshold)
        rowThres=np.nonzero(rowMeans > threshold)

        minCol=colThres[0][0]
        minRow=rowThres[0][0]
        maxCol=colThres[0][colThres[0].shape[0]-1]
        maxRow=rowThres[0][rowThres[0].shape[0]-1]

        imageX=np.delete(imageX, slice(maxCol,imageXGray.shape[1]), axis=1)
        imageX=np.delete(imageX, slice(0,minCol), axis=1)
        imageX=np.delete(imageX, slice(maxRow,imageXGray.shape[0]), axis=0)
        imageX=np.delete(imageX, slice(0,minRow), axis=0) 
        
        return imageX
    
    def setDefaultValues(self):
        hMin = 0   # Hue minimum
        sMin = 0   # Saturation minimum
        vMin = 0  # Brightness minimum
        hMax = 255 # Hue maximum
        sMax = 255 # Saturation maximum
        vMax = 255 # Value maximum

        self.lower = np.array([hMin, sMin, vMin])
        self.upper = np.array([hMax, sMax, vMax])
            
    def resize_image(self, img):
        scale = 20
        width = int(img.shape[1]*scale/100)
        height = int(img.shape[0]*scale/100)

        dimensions = width, height

        return cv2.resize(img, dimensions, interpolation = cv2.INTER_AREA)

    def remove_black_bg(self, img):
        self.setDefaultValues()
        
        hsvImage = cv2.cvtColor(img, cv2.COLOR_BGR2HSV) 
        mask = cv2.inRange(hsvImage, self.lower, self.upper) 
        output = cv2.bitwise_and(img, img, mask=mask)
        *_, alpha = cv2.split(output)

        mergedImage = cv2.merge((output, alpha))
        
        #outputImage = cv2.cvtColor(mergedImage, cv2.COLOR_LAB2RGB)
        
        return mergedImage

In [247]:
removeBg = removeBlackBackground()

print('Processing')

directory = 'testTubes'

for filename in os.listdir(directory):
    if filename.endswith(".jpg"):
        
        #Read image
        path = directory + '/' + filename
        img = cv2.imread(path)
        img_gray = cv2.imread(path, 0)
        
        img_resized = removeBg.resize_image(img)
        img_gray_resized = removeBg.resize_image(img_gray)
        
        removedBlankSpaceImg = removeBg.cropImage(img_resized, img_gray_resized, 50, 255, 5)
        
        #Remove background
        editedImg = removeBg.remove_black_bg(removedBlankSpaceImg)
        
        outputImg = editedImg
        
        cv2.imwrite('testTubesOutput/edited_'+filename[:-4]+'.png', outputImg)        
        continue
    else:
        continue
        
print('End of processing')

Processing
End of processing


## Create more images
Flip and transform the images

In [241]:
directory = 'testTubes'

print('Processing')

for filename in os.listdir(directory):
    if (filename.endswith('.jpg')) and not(fnmatch.fnmatch(filename, 'rotated*')) and not(fnmatch.fnmatch(filename, 'flipped*')):
        
        #Read image
        path = directory + '/' + filename
        img = cv2.imread(path)
        
        #rotate 90 degrees
        rotatedImg = np.rot90(img)
        cv2.imwrite('testTubes/rotated90_'+filename[:-4]+'.jpg', rotatedImg)
        
        #flip image
        flippedImg = np.flip(img)
        cv2.imwrite('testTubes/flipped_'+filename[:-4]+'.jpg', flippedImg)
        
print('End of processing')

Processing
End of processing
